---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [184]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [185]:
# from IPython.core.display import HTML
# css = open('style-table.css').read() + open('style-notebook.css').read()
# HTML('<style>{}</style>'.format(css))
# #to change the style of the panda dataframes. make it clearer to view content and headers

### Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [186]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [187]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df1 = pd.read_table('university_towns.txt', header = None) #initialization
    state = []; regionname = [] #create empty lists to assign later to columns
    for row in df1.itertuples(): #iterate through the dataframe
        if row[1].rstrip().endswith('edit]'): #check which rows are states
            state.append(row[1].rstrip().split('[')[0]) #insert state in clean format
            regionname.append('') #add empty region
        else:
            state.append(np.nan) #fill with nan the ones that are regions; later cleanup
            regionname.append(row[1].rstrip().split('(')[0].rstrip()) #insert region in clean format

    df1['State'] = state; df1['RegionName'] = regionname #assign the values of the lists to the dataframe
    df1 = df1.fillna(method = 'ffill') #fill the empty states of the regions

    df1 = df1[df1[0].str.endswith('edit]') != True].drop(0, axis = 1) #stay only with regions, delete the original index/column
    df1 = df1.reset_index().drop('index', axis = 1)
    return df1
# get_list_of_university_towns()

In [188]:
# # troubleshooting

# df = get_list_of_university_towns()
# cols = ["State", "RegionName"]

# print('Shape test: ', "Passed" if df.shape ==
#       (517, 2) else 'Failed')
# print('Index test: ',
#       "Passed" if df.index.tolist() == list(range(517))
#       else 'Failed')

# print('Column test: ',
#       "Passed" if df.columns.tolist() == cols else 'Failed')
# print('\\n test: ',
#       "Failed" if any(df[cols[0]].str.contains(
#           '\n')) or any(df[cols[1]].str.contains('\n'))
#       else 'Passed')
# print('Trailing whitespace test:',
#       "Failed" if any(df[cols[0]].str.contains(
#           '\s+$')) or any(df[cols[1]].str.contains(
#           '\s+$'))
#       else 'Passed')
# print('"(" test:',
#       "Failed" if any(df[cols[0]].str.contains(
#           '\(')) or any(df[cols[1]].str.contains(
#           '\('))
#       else 'Passed')
# print('"[" test:',
#       "Failed" if any(df[cols[0]].str.contains(
#           '\[')) or any(df[cols[1]].str.contains(
#           '\]'))
#       else 'Passed')
# # from troubleshooting learners code, using incorrect regex
# # for RegionName usually generates these common mismatches
# rgn_rgx_mstch = [
#     'Pomona', 'Mankato', 'Fulton', 'Sewanee']
# rgn_loc = (33, 218, 237, 442)
# print ('RegionName regex test: ', "Passed" if all(df.loc[
#        rgn_loc, 'RegionName'] == rgn_rgx_mstch)
#        else "Failed")
# # when using split or find to extract regionName, these are
# # common mismatches
# rgn_splt_msmtch = [
#     'The Five College Region of Western Massachusetts:',
#     'Faribault, South Central College']
# rgn_loc = (184, 217)
# print ('RegionName regex test: ', "Passed" if all(df.loc[
#        rgn_loc, 'RegionName'] == rgn_splt_msmtch)
#        else "Failed")
# #test if . was  inadvertently removed
# print('RegionName regex test:',"Passed" if any(df[cols[1]].str.contains(
#           '\.')) else "Failed")
# # use the values of the states dictionary precoded to verify
# # state names are as expected
# states_vlist = list(sorted(states.values()))
# mismatchedStates = df[~df['State'].isin(
#     states_vlist)].loc[:, 'State'].unique()
# print ('State regex test: ', "Passed" if len(
#     mismatchedStates) == 0 else "Failed")
# if len(mismatchedStates) > 0:
#     print()
#     print('The following states failed the equality test:')
#     print()
#     print('\n'.join(mismatchedStates))

In [189]:
def load_gdp():
    gdp = pd.read_excel('gdplev.xls', skiprows = 7, names = ['year_quarter', 'gdpbillions2009dol'], parse_cols = [4, 6]) #load data
    gdp = gdp.loc[gdp[gdp['year_quarter'] == '2000q1'].index.tolist()[0]:len(gdp)].reset_index(drop = True)
    #find the value where 2000 starts, use this as the starting point
    return gdp
# load_gdp()

In [194]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    #     initial data loading & handling
    recstart = load_gdp()
    recstart['gdp_b'] = [1 if x > 0 else 0 for x in recstart['gdpbillions2009dol'].diff()] #get the current value - the before value
    quarters = []; befores = -1

    #iterate on the values of the differences, to start a recession two continuous decrease periods should happen
    for i, v in recstart['gdp_b'].iteritems():
    #     print(i, v, befores) #debugging
        if (v == befores) & (v == 0): #current period is the same as the before period (and it's a decrease)
            quarters.append(i-1) #the recession started the period before of the 2nd decrease happening
            break
        befores = v

    return recstart['year_quarter'].loc[quarters[0]]
# get_recession_start()

'2008q3'

In [206]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''       
    #recession start information
    recstart = load_gdp()
    recstart['gdp_b'] = [1 if x > 0 else 0 for x in recstart['gdpbillions2009dol'].diff()] #get the current value - the before value
    quarters = []; befores = -1
    for i, v in recstart['gdp_b'].iteritems():
        if (v == befores) & (v == 0):
            quarters.append(i-1)
            break
        befores = v
    
    
    #initial data loading & handling
    recend = load_gdp().loc[quarters[0]:]
    recend['gdp_b'] = [1 if x > 0 else 0 for x in recend['gdpbillions2009dol'].diff()] #get the current value - the before value
    quartere = []; beforee = -1
    #iterate on the values of the differences, to end a recession two continuous decrease periods
    #should happen after the start period
    for i, v in recend['gdp_b'].iteritems():
        #     print(i, v, before) #debugging
        if (v == beforee) & (v == 1): #current period is the same as the before period (and it's a decrease)
            quartere.append(i) #the recession ended the period when the 2nd increment happened
            break
        beforee = v

    return recend['year_quarter'].loc[quartere[0]]
get_recession_end()

'2009q4'

In [207]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    #recession start information
    recstart = load_gdp()
    recstart['gdp_b'] = [1 if x > 0 else 0 for x in recstart['gdpbillions2009dol'].diff()] #get the current value - the before value
    quarters = []; befores = -1
    for i, v in recstart['gdp_b'].iteritems():
        if (v == befores) & (v == 0):
            quarters.append(i-1)
            break
        befores = v
    
    #recession end information
    recend = load_gdp().loc[quarters[0]:]
    recend['gdp_b'] = [1 if x > 0 else 0 for x in recend['gdpbillions2009dol'].diff()] #get the current value - the before value
    quartere = []; beforee = -1
    for i, v in recend['gdp_b'].iteritems():
        if (v == beforee) & (v == 1):
            quartere.append(i)
            break
        beforee = v
    
    #getting the bottom recession information
    recbot = load_gdp().loc[quarters[0]:quartere[0]] #filter by start and end of recession
    botind = recbot[recbot['gdpbillions2009dol'] == recbot['gdpbillions2009dol'].min()].index.tolist()#get the min, get index
    return recbot['year_quarter'].loc[botind[0]]
# get_recession_bottom()

'2009q2'

In [234]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    rawhdf = pd.read_csv('City_Zhvi_AllHomes.csv')
    rawhdf = rawhdf.drop(rawhdf.columns[[0]+list(range(3,51))],axis=1) #only get state, region and 2000 and later
    hdf = pd.DataFrame(rawhdf[['State','RegionName']]) #new clean df, to start building quarters
    #build the quarters, 3 months each quarter
    for year in range(2000,2016):
        hdf[str(year)+'q1'] = rawhdf[[str(year)+'-01',str(year)+'-02',str(year)+'-03']].mean(axis=1)
        hdf[str(year)+'q2'] = rawhdf[[str(year)+'-04',str(year)+'-05',str(year)+'-06']].mean(axis=1)
        hdf[str(year)+'q3'] = rawhdf[[str(year)+'-07',str(year)+'-08',str(year)+'-09']].mean(axis=1)
        hdf[str(year)+'q4'] = rawhdf[[str(year)+'-10',str(year)+'-11',str(year)+'-12']].mean(axis=1)

    year = 2016    
    hdf[str(year)+'q1'] = rawhdf[[str(year)+'-01',str(year)+'-02',str(year)+'-03']].mean(axis=1)
    hdf[str(year)+'q2'] = rawhdf[[str(year)+'-04',str(year)+'-05',str(year)+'-06']].mean(axis=1)
    hdf[str(year)+'q3'] = rawhdf[[str(year)+'-07',str(year)+'-08']].mean(axis=1)
    hdf = hdf.replace({'State':states}) #change the states initials for the proper name
    hdf = hdf.set_index(['State','RegionName'])
    return hdf
# convert_housing_data_to_quarters()

In [235]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    #prepare data
    unitowns = get_list_of_university_towns()
    bottom = get_recession_bottom()
    start = get_recession_start()
    hdata = convert_housing_data_to_quarters()
    bstart = hdata.columns[hdata.columns.get_loc(start) -1]
    
    hdata['ratio'] = hdata[bottom] - hdata[bstart]
    hdata = hdata[[bottom,bstart,'ratio']]
    hdata = hdata.reset_index()
    unitowns_hdata = pd.merge(hdata,unitowns,how='inner',on=['State','RegionName'])
    unitowns_hdata['uni'] = True
    hdata2 = pd.merge(hdata,unitowns_hdata,how='outer',on=['State','RegionName',bottom,bstart,'ratio'])
    hdata2['uni'] = hdata2['uni'].fillna(False)

    ut = hdata2[hdata2['uni'] == True]
    nut = hdata2[hdata2['uni'] == False]

    t,p = ttest_ind(ut['ratio'].dropna(),nut['ratio'].dropna())
    
    different = True if p < 0.01 else False

    better = "non-university town" if ut['ratio'].mean() < nut['ratio'].mean() else "university town"

    return different, p, better
# run_ttest()

(True, 0.002099659657952052, 'university town')